# Accessing ACME
### Ray Smith

In [57]:
# Imports
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

import time

In [30]:
# Settings
netid_utility_url = "https://kb.wisc.edu/helpdesk/internal/tools/netid/index.php"
acme_url = "https://acme.wisc.edu/tools/schedule/schedule.php"

login_secret = "login.secret"
duo_secret = "duo.secret"

options = Options()
# options.headless = True

In [31]:
# Get Secrets
with open(login_secret) as f:
    user = f.readline()[:-1]
    password = f.readline()
    
with open(duo_secret) as f:
    duo = f.readline()

In [64]:
# Login to ACME
browser = webdriver.Chrome(options=options)
browser.get(url)

# Fill in username
user_field = browser.find_element_by_id('j_username')
user_field.send_keys(user)

# Fill in password
pass_field = browser.find_element_by_id('j_password')
pass_field.send_keys(password)

# Click login
login_button = browser.find_element_by_name('_eventId_proceed')
login_button.click()

# Wait for next page to load
delay = 3
time.sleep(delay)

# Move to within the iFrame
iframe = driver.find_elements_by_tag_name('iframe')[0]
browser.switch_to_frame(iframe)

# Enter a duo passcode
passcode_button = browser.find_element_by_id('passcode')
passcode_button.click()

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=57050): Max retries exceeded with url: /session/ba045223e4a04f0ad39ef791a3753358/elements (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x10d4fc4e0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [36]:
# Show page source
# print(browser.page_source)

In [63]:
# Close browser
browser.close()